# Chapter 11 - 인터페이스: 프로토콜에서 ABC까지

## 파이썬 문화에서의 인터페이스와 프로토콜

인터페이스: 클래스가 상속하거나 구현한 공개 속성(메서드나 데이터 속성)들의 집합

In [1]:
class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
    
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    # ...

인터페이스의 정의에 대해서는 '시스템에서 어떤 역할을 할 수 있게 해주는 객체의 공개 메서드의 일부'라고 설명을 보충할 수 있다.<br>
바로 이것이 파이썬 문서에서 특정 클래스를 지정하지 않고 'file과 같은 객체' 혹은 '반복형'이라고 말할 때 의미하는 것이다.<br>
어떤 역할을 완수하기 위한 메서드 집합으로써의 인터페이스를 스몰토크에서는 **프로토콜**이라고 불렀으며, 이 용어는 다른 동적 언어 커뮤니티에도 퍼져나갔다.<br>
<br>
프로토콜은 인터페이스지만 비공식적이다.<br>
즉, 문서와 관례에 따라 정의되지만 공식 인터페이스처럼 강제할 수 없다.<br>
프로토콜은 특정 클래스에서 부분적으로 구현할 수 있으며, 이렇게 해도 문제가 없다.

## 파이썬은 시퀀스를 찾아낸다

![](https://i.ibb.co/nfsF3YK/asdf.png)<br>

위 그림은 ABC로 정의된 공식적인 시퀀스 인터페이스를 보여준다.

In [3]:
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

f = Foo()
f[1]

10

In [4]:
for i in f:
    print(i)

0
10
20


In [5]:
20 in f

True

Foo 클래스는 abc.Sequence를 상속하지 않으며 시퀀스 프로토콜 메서드 중 \_\_getitem\_\_() 메서드 하나만 구현한다.<br>
\_\_iter\_\_() 메서드는 아직 구현하지 않았지만, 대체 수단인 \_\_getitem\_\_() 메서드가 구현되어 있으므로 Foo 객체를 반복할 수 있다.<br>
파이썬 인터프리터는 0부터 시작하는 정수 인덱스로 \_\_getitem\_\_() 메서드를 호출하여 객체 반복을 시도하기 때문이다.

## 런타임에 프로토콜을 구현하는 멍키 패칭

멍키 패칭: 소스 코드를 건드리지 않고 런타임에 클래스나 모듈을 변경하는 행위.

In [2]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]
        
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    

In [3]:
from random import shuffle

l = list(range(10))
shuffle(l)
l

[1, 6, 7, 3, 8, 2, 4, 9, 5, 0]

In [4]:
deck = FrenchDeck()
shuffle(deck)

TypeError: 'FrenchDeck' object does not support item assignment

FrenchDeck 객체가 할당을 지원하지 않기 때문에 shuffle 함수의 인자로 넘겨줄 수 없다.<br>
FrenchDeck 클래스는 불변 시퀀스 프로토콜만 구현하고 있으므로, \_\_setitem\_\_() 메서드도 지원해야 한다.

In [5]:
def set_card(deck, position, card):
    deck._cards[position] = card
    
FrenchDeck.__setitem__ = set_card
shuffle(deck)
deck[:5]

[Card(rank='2', suit='spades'),
 Card(rank='8', suit='clubs'),
 Card(rank='A', suit='spades'),
 Card(rank='7', suit='spades'),
 Card(rank='K', suit='spades')]